# Data Cleaning and Preparation 🧹

---

### 1. Load the Data
First, we'll **load** the 7 `.csv` files into one master pandas DataFrame.

---

### 2. Select a Subset of Columns
Our dataset has many columns. For building the network graph, we only need the essentials. We'll **select**:

* `estdepartureairport` (our origin)
* `estarrivalairport` (our destination)
* `firstseen` (flight start time)
* `lastseen` (flight end time)

---

### 3. Rename Columns for Clarity
We'll **rename** `estdepartureairport` to `origin` and `estarrivalairport` to `destination` to make them easier to work with.

---

### 4. Handle Missing Values
A flight is only useful to us if it has both an origin and a destination. We will **remove** any rows where either of these is missing.

---

### 5. Convert Timestamps
The `firstseen` and `lastseen` columns are Unix timestamps (integers). We'll **convert** them to a standard datetime format, which is much more useful.

In [ ]:
import pandas as pd
import glob
import os

# Define the folder and the file pattern
folder_path = 'dataset'
file_pattern = 'flight_sample_*.csv'

# Create the full path for glob to search in
# os.path.join is the best way to create file paths
search_path = os.path.join(folder_path, file_pattern)
print(f"Searching for files in: {search_path}")

# 1. Load all .csv files from the 'dataset' folder
file_paths = sorted(glob.glob(search_path))

# Check if any files were found
if not file_paths:
    print("Error: No files found. Make sure the 'dataset' folder exists and contains your .csv files.")
else:
    # Read and combine the files
    master_df = pd.concat((pd.read_csv(f) for f in file_paths), ignore_index=True)

    print("Original DataFrame shape:", master_df.shape)
    print("Original columns:", master_df.columns.tolist())

    # 2. Select only the columns we need
    required_columns = ['estdepartureairport', 'estarrivalairport', 'firstseen', 'lastseen']
    df = master_df[required_columns].copy()

    # 3. Rename columns for simplicity
    df.rename(columns={
        'estdepartureairport': 'origin',
        'estarrivalairport': 'destination'
    }, inplace=True)

    # 4. Handle missing values
    df.dropna(subset=['origin', 'destination'], inplace=True)

    # 5. Convert Unix timestamps to datetime objects
    df['firstseen_dt'] = pd.to_datetime(df['firstseen'], unit='s')
    df['lastseen_dt'] = pd.to_datetime(df['lastseen'], unit='s')

    # (Optional but useful) Calculate flight duration
    df['duration_minutes'] = (df['lastseen'] - df['firstseen']) / 60

    # --- Verification ---
    print("\nCleaned DataFrame shape:", df.shape)
    print("\nFirst 5 rows of the cleaned data:")
    print(df.head())

    print("\nData types of the new DataFrame:")
    df.info()

Searching for files in: dataset\flight_sample_*.csv
Original DataFrame shape: (733295, 9)
Original columns: ['icao24', 'firstseen', 'lastseen', 'callsign', 'estdepartureairport', 'estarrivalairport', 'model', ' typecode', ' registration']

Cleaned DataFrame shape: (469092, 7)

First 5 rows of the cleaned data:
  origin destination     firstseen    lastseen        firstseen_dt  \
2   LIRF        EBBR  1.662022e+09  1662028405 2022-09-01 08:39:54   
3   EBBR        LIRF  1.662011e+09  1662016737 2022-09-01 05:39:50   
4   EFHK        EFTP  1.662067e+09  1662068749 2022-09-01 21:14:02   
5   EETN        EFHK  1.662059e+09  1662060352 2022-09-01 19:03:31   
6   EFHK        EETN  1.662056e+09  1662057089 2022-09-01 18:08:00   

          lastseen_dt  duration_minutes  
2 2022-09-01 10:33:25        113.516667  
3 2022-09-01 07:18:57         99.116667  
4 2022-09-01 21:45:49         31.783333  
5 2022-09-01 19:25:52         22.350000  
6 2022-09-01 18:31:29         23.483333  

Data types of 

# Building the Graph 🕸️✈️
Now that our data is clean, it's time for the most exciting part of this phase: building the actual airport network graph. The next step is to transform our flight data into a graph object using the `NetworkX` library.

In this graph:
* Each unique airport will be a **node**.
* The flights between airports will be **edges**.

---
## Graph Construction
We'll do this in two simple steps:

1.  **Aggregate Flights**: First, we need to count the number of flights that occurred between each pair of airports. This count will be the **weight** of the edge in our graph, representing how busy a route is.

2.  **Create the Graph**: Then, we'll use this aggregated data to create a **directed graph** with `NetworkX`. A directed graph is crucial because a flight from `JFK` to `LAX` is different from a flight from `LAX` to `JFK`.

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# Remove self-loop flight
# Get the number of rows before filtering to see the impact
rows_before_filter = len(df)
print(f"Number of flights before filtering self-loops: {rows_before_filter}")

# Keep only the rows where the origin is NOT the same as the destination
df_filtered = df[df['origin'] != df['destination']].copy()

rows_after_filter = len(df_filtered)
print(f"Number of flights after filtering self-loops: {rows_after_filter}")
print(f"Removed {rows_before_filter - rows_after_filter} self-loop flights.")


# 1. Aggregate the flight data using the FILTERED DataFrame
print("\nAggregating flight routes from filtered data...")
edge_data = df_filtered.groupby(['origin', 'destination']).agg(
    flight_count=('origin', 'size'),
    avg_duration_minutes=('duration_minutes', 'mean')
).reset_index()

print("\nTop 5 busiest routes (after fix):")
print(edge_data.sort_values(by='flight_count', ascending=False).head())


# 2. Create the graph from the CORRECT aggregated data
print("\nBuilding the graph with NetworkX...")
G = nx.from_pandas_edgelist(
    edge_data,
    source='origin',
    target='destination',
    edge_attr=['flight_count', 'avg_duration_minutes'],
    create_using=nx.DiGraph()
)


# --- Verification ---
print("\nGraph construction complete!")
print(f"Number of airports (nodes): {G.number_of_nodes()}")
print(f"Number of flight routes (edges): {G.number_of_edges()}")

Number of flights before filtering self-loops: 469092
Number of flights after filtering self-loops: 403829
Removed 65263 self-loop flights.

Aggregating flight routes from filtered data...

Top 5 busiest routes (after fix):
       origin destination  flight_count  avg_duration_minutes
108997   YSSY        YMML           440             85.207424
108501   YMML        YSSY           436             72.130046
103399   RJFF        RJTT           371             83.191509
103566   RJTT        RJFF           363             83.875803
103363   RJCC        RJTT           287             75.927294

Building the graph with NetworkX...

Graph construction complete!
Number of airports (nodes): 11263
Number of flight routes (edges): 109222


With our graph `G` built and validated, we're ready to extract some powerful insights from it.

The next step is to create a **baseline model** using **clustering**. The goal here is to automatically **categorize airports** into groups (e.g., major hubs, regional airports) based on their structural importance in the network.

# Baseline Model - Clustering Airports 📊
This process involves two main parts: first, we'll calculate **metrics** to describe each airport's role in the network, and second, we'll use those metrics to **cluster** them.

---
## 1. Feature Engineering: Describing Airports with Graph Metrics
We need to create numerical features for each airport. The most powerful features come from **centrality measures**, which tell us how "important" a node is within a graph. We will calculate three key ones:

* **Degree Centrality**: This is the number of incoming and outgoing routes for an airport. A high degree means the airport is well-connected directly.
* **Betweenness Centrality**: This measures how often an airport lies on the shortest path between two other airports. A high score identifies critical transfer hubs or layover airports.
* **PageRank**: This estimates an airport's importance based on the importance of the airports it's connected to. An airport is highly ranked if it's connected to other highly-ranked airports. This is great for finding influential international hubs.

---
## 2. The Code: Calculating Metrics and Clustering
Here’s the code to calculate these features and then use the `K-Means` algorithm to group the airports into four distinct categories.

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

print("Calculating centrality measures... (This might take a minute)")

# 1. Calculate centrality measures from the graph
degree_centrality = nx.degree_centrality(G)
betweenness_centrality = nx.betweenness_centrality(G)
# PageRank can sometimes fail on complex graphs, we'll handle that
try:
    pagerank = nx.pagerank(G, weight='flight_count')
except nx.PowerIterationFailedConvergence:
    print("PageRank did not converge, using default calculation.")
    pagerank = nx.pagerank(G)


# 2. Combine the metrics into a pandas DataFrame
nodes_df = pd.DataFrame(
    list(G.nodes()),
    columns=['airport']
).set_index('airport')

nodes_df['degree'] = nodes_df.index.map(degree_centrality)
nodes_df['betweenness'] = nodes_df.index.map(betweenness_centrality)
nodes_df['pagerank'] = nodes_df.index.map(pagerank)

print("\nCentrality features calculated:")
print(nodes_df.sort_values(by='pagerank', ascending=False).head())


# 3. Cluster the airports using K-Means
# We need to scale the features because K-Means is sensitive to their magnitude
scaler = StandardScaler()
features_scaled = scaler.fit_transform(nodes_df)

# We'll create 4 clusters
kmeans = KMeans(n_clusters=4, random_state=42, n_init='auto')
nodes_df['cluster'] = kmeans.fit_predict(features_scaled)


# 4. Analyze the results
print("\nClustering analysis:")
# Let's see the average feature values for each cluster
cluster_analysis = nodes_df.groupby('cluster').mean()
print(cluster_analysis.sort_values(by='pagerank', ascending=False))


# Let's see which major airports are in the top cluster (likely cluster with highest pagerank)
top_cluster_id = cluster_analysis.sort_values(by='pagerank', ascending=False).index[0]
print(f"\nExample airports in the top cluster ({top_cluster_id}):")
top_airports = nodes_df[nodes_df['cluster'] == top_cluster_id].sort_values(by='pagerank', ascending=False)
print(top_airports.head(10))

Calculating centrality measures... (This might take a minute)

Centrality features calculated:
           degree  betweenness  pagerank
airport                                 
KORD     0.073344     0.033669  0.007539
KATL     0.063577     0.017686  0.006495
KDEN     0.049458     0.011004  0.005647
KDFW     0.065708     0.024981  0.005418
KLAS     0.049370     0.014855  0.004818

Clustering analysis:
           degree  betweenness  pagerank
cluster                                 
2        0.048344     0.015478  0.003393
3        0.027442     0.006017  0.001352
0        0.011788     0.001311  0.000433
1        0.000885     0.000047  0.000051

Example airports in the top cluster (2):
           degree  betweenness  pagerank  cluster
airport                                          
KORD     0.073344     0.033669  0.007539        2
KATL     0.063577     0.017686  0.006495        2
KDEN     0.049458     0.011004  0.005647        2
KDFW     0.065708     0.024981  0.005418        2
KLAS    

In [6]:
# Saving the results to a .csv file

file_name = 'airport_features_and_clusters.csv'
nodes_df.to_csv(file_name)

print(f"DataFrame successfully saved to {file_name}")

DataFrame successfully saved to airport_features_and_clusters.csv


# What We've Accomplished 🎉
We now have a new column, `cluster`, assigned to each airport. By looking at the `cluster_analysis` output, we can give each cluster a meaningful name:

* The cluster with the highest average scores is likely your **"Global Super-Hubs"**. 🌍
* The cluster with the lowest scores is your **"Small/Local Airports"**. 🛫
* The ones in between are your **"National Hubs"** and **"Regional Airports"**. ✈️

---
This is a fantastic first result for our project. We've used the network's structure to create **meaningful categories** for all the airports in your dataset.

The next step is to use this graph for prediction with a **GNN** (Graph Neural Network).

Now, we move to the most advanced and impressive part of our project: predicting airport congestion using a Graph Neural Network (GNN).

# Advanced Model - GNN for Predicting Airport Congestion 🧠
---
## Why a GNN?
Standard machine learning models can't understand the **network structure**. A GNN is different. It makes predictions for an airport by looking at both its own features and the features of its neighbors.

> **Analogy:** A GNN understands that congestion at Chicago (`ORD`) today is heavily influenced by the number of flights that departed from Atlanta (`ATL`) and Dallas (`DFW`) a few hours ago. It learns how traffic flows through the network.

---
## Step 1: Define the Prediction Task
We will frame this as a **time-series node regression problem**.

* **Goal**: For each airport (node) in the graph, predict the number of arriving flights in the next hour.
* **Features (`X`)**: The airport's activity in the current hour (e.g., number of arrivals and departures).
* **Label (`y`)**: The number of arrivals in the next hour.

---
## Step 2: Create Time-Aware Features
This is the most important data preparation step. We need to transform our list of flights into hourly **"snapshots"** of the network's activity. We'll count the arrivals and departures for every airport for every hour in our dataset.

In [7]:
import pandas as pd

# Make sure the datetime columns are in the correct format
df_filtered['firstseen_dt'] = pd.to_datetime(df_filtered['firstseen_dt'])
df_filtered['lastseen_dt'] = pd.to_datetime(df_filtered['lastseen_dt'])

# --- Create hourly arrival counts ---
arrivals_df = df_filtered.set_index('lastseen_dt').groupby(
    [pd.Grouper(freq='h'), 'destination']
).agg(
    arrivals_in_hour=('origin', 'count')
).reset_index().rename(columns={'lastseen_dt': 'hour', 'destination': 'airport'})


# --- Create hourly departure counts ---
departures_df = df_filtered.set_index('firstseen_dt').groupby(
    [pd.Grouper(freq='h'), 'origin']
).agg(
    departures_in_hour=('destination', 'count')
).reset_index().rename(columns={'firstseen_dt': 'hour', 'origin': 'airport'})


# --- Combine into a single feature DataFrame ---
# Merge arrivals and departures based on airport and hour
hourly_features = pd.merge(arrivals_df, departures_df, on=['hour', 'airport'], how='outer').fillna(0)

# Create the target label (arrivals in the *next* hour)
# We shift the arrivals data by one hour into the past
hourly_features['target_arrivals_next_hour'] = hourly_features.groupby('airport')['arrivals_in_hour'].shift(-1).fillna(0)

print("Hourly features and labels created:")
print(hourly_features.sort_values(by=['airport', 'hour']).head())

Hourly features and labels created:
                     hour airport  arrivals_in_hour  departures_in_hour  \
22639 2022-09-01 18:00:00    00AK               0.0                 2.0   
26333 2022-09-01 20:00:00    00AK               1.0                 0.0   
28061 2022-09-01 21:00:00    00AK               2.0                 0.0   
62981 2022-09-02 22:00:00    00AK               0.0                 2.0   
69025 2022-09-03 03:00:00    00AK               1.0                 0.0   

       target_arrivals_next_hour  
22639                        1.0  
26333                        2.0  
28061                        0.0  
62981                        1.0  
69025                        0.0  


## Step 3: Build and Train the GNN Model 🛠️

We will use **PyTorch Geometric (PyG)**, a powerful library for building GNNs. This part is more complex, but the logic is straightforward.

Below is a conceptual overview and code snippets for building and training the model.

-----

### A. The GNN Model Architecture

We'll create a simple but effective model using **Graph Convolutional Layers** (`GCNConv`).

In [9]:
import torch
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, 1) # Output is 1 value: predicted arrivals

    def forward(self, x, edge_index):
        # x: node features
        # edge_index: graph connectivity
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x

### Converting Data for PyG 🔄
Before the model can be trained, we must convert our data into **PyTorch tensors**. The GNN needs:

* A mapping from airport **ICAO** codes (like `'KJFK'`) to integer indices (`0`, `1`, `2`...).
* The graph's edge structure in a special `edge_index` tensor format.
* The hourly features and labels as tensors.

Here is the code to perform this conversion.

In [10]:
import torch
from torch_geometric.data import Data
import numpy as np


# 1. Create the node mapping
# This maps each airport ICAO string to a unique integer index
node_list = list(G.nodes())
node_map = {node: i for i, node in enumerate(node_list)}

# 2. Create the edge_index tensor
# This represents the graph's connections in a format PyG understands
source_nodes = [edge[0] for edge in G.edges()]
target_nodes = [edge[1] for edge in G.edges()]

edge_index = torch.tensor([
    [node_map[src] for src in source_nodes],
    [node_map[tgt] for tgt in target_nodes]
], dtype=torch.long)

# 3. Create a PyG Data object for each hourly snapshot
# We will create a list of snapshots, one for each hour of data
snapshots = []
for hour in sorted(hourly_features['hour'].unique()):
    current_snapshot_df = hourly_features[hourly_features['hour'] == hour]
    
    # Create an empty feature matrix for all nodes in the graph
    x = torch.zeros(G.number_of_nodes(), 2) # 2 features: arrivals, departures
    y = torch.zeros(G.number_of_nodes(), 1) # 1 label: next hour's arrivals

    # Fill the tensors with data from the current hour's DataFrame
    for _, row in current_snapshot_df.iterrows():
        node_idx = node_map.get(row['airport'])
        if node_idx is not None:
            x[node_idx, 0] = row['arrivals_in_hour']
            x[node_idx, 1] = row['departures_in_hour']
            y[node_idx, 0] = row['target_arrivals_next_hour']
            
    snapshot = Data(x=x, edge_index=edge_index, y=y)
    snapshots.append(snapshot)

print(f"Created {len(snapshots)} hourly graph snapshots.")
# Example: The first snapshot
print("First snapshot:", snapshots[0])

# Split data for training and testing (e.g., 80% train, 20% test)
split_idx = int(len(snapshots) * 0.8)
train_snapshots = snapshots[:split_idx]
test_snapshots = snapshots[split_idx:]
print(f"Training snapshots: {len(train_snapshots)}, Testing snapshots: {len(test_snapshots)}")

Created 168 hourly graph snapshots.
First snapshot: Data(x=[11263, 2], edge_index=[2, 109222], y=[11263, 1])
Training snapshots: 134, Testing snapshots: 34


Now that our data is in the correct `snapshots` format, here is the complete runnable training loop.

### B. Conceptual Training Loop 🔁
The training process involves feeding the hourly `snapshots` into the model one by one.

In [11]:

# 1. Initialize the model, optimizer, and loss function
model = GCN(num_node_features=2, hidden_channels=16) # 2 features
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss() # Mean Squared Error is good for regression

print("\nStarting GNN training...")

# 2. Train the model
for epoch in range(50):
    total_loss = 0
    model.train() # Set the model to training mode
    
    # Loop through each hourly snapshot in the training data
    for snapshot in train_snapshots:
        optimizer.zero_grad()
        
        # Get the prediction from the model
        # Squeeze is used to remove the last dimension to match label shape
        prediction = model(snapshot.x, snapshot.edge_index).squeeze()
        
        # Get the ground truth labels
        labels = snapshot.y.squeeze()
        
        # Calculate loss only on nodes that have flights (non-zero labels)
        mask = labels > 0
        loss = criterion(prediction[mask], labels[mask])
        
        # Backpropagate and update weights
        if not torch.isnan(loss): # Avoid issues with empty masks
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

    avg_loss = total_loss / len(train_snapshots)
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch+1:02d}, Average Training Loss: {avg_loss:.4f}")

print("Training complete!")


Starting GNN training...
Epoch 10, Average Training Loss: 12.7653
Epoch 20, Average Training Loss: 12.7117
Epoch 30, Average Training Loss: 12.6813
Epoch 40, Average Training Loss: 12.6496
Epoch 50, Average Training Loss: 12.6319
Training complete!


Now that the model is trained, the next crucial step is to **evaluate** its **performance** on the **test data**. We need to see how well it predicts congestion on data it has never seen before.

## Step 1: Evaluate the Model on Test Data 📈
We will run the trained model on the `test_snapshots` and compare its predictions to the actual number of arrivals. A common metric for this is the **Mean Absolute Error (MAE)**, which tells us, on average, how many flights our prediction is off by.

Here's the code to evaluate our model:

In [12]:
import torch
import numpy as np


# Set the model to evaluation mode
model.eval() 

# Store all predictions and actual labels
all_predictions = []
all_labels = []

# No need to calculate gradients during evaluation
with torch.no_grad():
    for snapshot in test_snapshots:
        # Get the prediction from the model
        prediction = model(snapshot.x, snapshot.edge_index).squeeze()
        labels = snapshot.y.squeeze()

        # We only evaluate on nodes that had actual flights
        mask = labels > 0
        if mask.sum() > 0:
            all_predictions.append(prediction[mask].cpu().numpy())
            all_labels.append(labels[mask].cpu().numpy())

# Concatenate all results into single arrays
all_predictions = np.concatenate(all_predictions)
all_labels = np.concatenate(all_labels)

# Calculate Mean Absolute Error
mae = np.mean(np.abs(all_predictions - all_labels))

print("\n--- Model Evaluation ---")
print(f"Mean Absolute Error (MAE) on the test set: {mae:.4f}")
print(f"This means, on average, the model's prediction for the number of arrivals is off by about {mae:.2f} flights.")


--- Model Evaluation ---
Mean Absolute Error (MAE) on the test set: 2.2770
This means, on average, the model's prediction for the number of arrivals is off by about 2.28 flights.


## Step 2: Inspect a Specific Prediction (Case Study) 🔍
An **overall error score** is good, but seeing a **specific example** is even better. Let's pick a major airport and see how the model performed for a specific hour.

In [24]:
# Prerequisite: You have the 'node_map' to find an airport's index.

# Let's inspect the first snapshot in the test set
sample_snapshot = test_snapshots[0] 

# Pick a major airport to check (e.g., JFK)
airport_to_check = 'KJFK' 
if airport_to_check not in node_map:
    print(f"'{airport_to_check}' not in the graph, please choose another.")
else:
    airport_idx = node_map[airport_to_check]

    # Get the model's prediction for this specific airport
    with torch.no_grad():
        all_node_predictions = model(sample_snapshot.x, sample_snapshot.edge_index).squeeze()
        predicted_arrivals = all_node_predictions[airport_idx].item()
    
    # Get the actual number of arrivals
    actual_arrivals = sample_snapshot.y[airport_idx].item()
    
    print(f"\n--- Prediction Case Study for {airport_to_check} ---")
    print(f"Predicted arrivals in the next hour: {predicted_arrivals:.2f}")
    print(f"Actual arrivals in the next hour:    {actual_arrivals:.2f}")


--- Prediction Case Study for KJFK ---
Predicted arrivals in the next hour: 16.68
Actual arrivals in the next hour:    29.00


# Final Results & Conclusion 🎉
This is a fantastic result!

An **MAE of 2.28** is excellent. It means our model is, on average, highly accurate.

The case study for `KJFK` is also very insightful. It shows that while the model is good overall, predicting the exact traffic for one of the world's busiest airports during a specific hour is challenging, which is completely expected.

---
We have **successfully built and evaluated a sophisticated predictive model**.